# Content-based model

Давайте реализуем подобную рекомендательную систему на практике. Будем работать с датасетом, содержащим информацию об оценивании фильмов на платформе Netflix

**Признаки данных:**

* show_id — id фильма,
* type — его тип (фильм или сериал),
* title — название,
* director — режиссер,
* cast — актерский состав,
* country — страна,
* release_year — год выхода на экраны,
* rating — рейтинг,
* duration — продолжительность,
* listened_in — жанр(-ы),
* description — описание.

В первую очередь нам необходимо определить, на основании чего мы будем рассматривать близость фильмов. Выберем для этой задачи описание фильма, ведь в нём, скорее всего, содержится много информации. Однако описание — это текст. Есть много подходов к преобразованию текста в вектор, и мы будем использовать подход `TF-IDF (Term Frequency-Inverse Document Frequency)`.

Показатель `TD-IDF` — это индикатор того, насколько релевантно слово в контексте документа.

Его можно определить следующим образом:
$\text{TF-IDF(слова) = TF(слова) * IDF (слова)}$

$\text{TF слова} = \frac{\text{Количество раз, когда слово встретилось в тексте}}{\text{Количество всех слов в тексте}}$

$\text{IDF слова} = log \left (\frac{\text{Общее кол-во документов}}{\text{Кол-во документов, в которых встречается слово}}\right )$

Этот показатель возрастает пропорционально количеству раз, когда слово встречается в тексте, и уменьшается пропорционально количеству слов во всех текстах в целом.

Таким образом:

* **Коэффициент будет выше**, если слово характерно именно для этого текста, то есть встречается в данном тексте часто, но не встречается в других текстах.
* **Коэффициент будет ниже**, если слово не встречается почти нигде или встречается одинаковое количество раз во всех текстах, то есть не характеризует никакой текст в отдельности.

In [1]:
#Чтобы преобразовать текст по этому принципу, нам понадобится соответствующая функция из библиотеки sklearn — импортируем её:
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
#Далее учтём стоп-слова, т. е. предлоги и другие служебные части речи, которые не несут содержательной информации, и с учётом этого определим нашу модель:
model = TfidfVectorizer(stop_words='english')

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('data/netflix_titles.zip')
#Заполним пропуски пустыми строками:
df['description'] = df['description'].fillna('')

In [5]:
#Трансформируем наши описания в матрицу:
feature_matrix = model.fit_transform(df['description'])

In [6]:
#Сколько столбцов в получившейся матрице?
feature_matrix.shape[1]
#17905

17905

In [7]:
#Теперь необходимо вычислить косинусную близость. Можно сделать это так:

from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(feature_matrix, feature_matrix)

**Обратите внимание!** Мы используем здесь `linear_kernel()`, а не `cosine_similarity()`, так как в косинусном расстоянии в знаменателе реализуется нормировка векторов, а `TF-IDF` создаёт уже нормализованные векторы.

In [8]:
#Вернём индексацию и уберём дубликаты из данных:
indices = pd.Series(df.index,index=df['title']).drop_duplicates()

In [9]:
#Теперь пропишем функцию для создания рекомендаций:

def get_recommendations(title):
    idx = indices[title]
    #вычисляем попарные коэффициенты косинусной близости
    scores = list(enumerate(cosine_sim[idx]))
    #сортируем фильмы на основании коэффициентов косинусной близости по убыванию
    scores = sorted(scores, key=lambda x: x[1], reverse=True)
    #выбираем десять наибольших значений косинусной близости; нулевую не берём, т. к. это тот же фильм
    scores =   scores[1:11]
    #забираем индексы
    ind_movie = [i[0] for i in scores]
    #возвращаем названия по индексам
    return df['title'].iloc[ind_movie]

In [10]:
#Например, если мы хотим найти рекомендации по фильму "Star Trek", то функция будет выдавать следующий результат:
get_recommendations('Star Trek')

5788             Star Trek: The Next Generation
5787                      Star Trek: Enterprise
5786                 Star Trek: Deep Space Nine
5557                     She's Out of My League
134                                  7 Days Out
6664                        The Midnight Gospel
6023                                     Teresa
4863    Pinkfong & Baby Shark's Space Adventure
5104                                       Rats
5970                             Tales by Light
Name: title, dtype: object

In [11]:
#Найдите вторую рекомендацию для детского фильма "Balto", вышедшего на экраны в 1995 году:
get_recommendations('Balto').iloc[1]
#Vroomiz

'Vroomiz'

# Коллаборативная фильтрация

В нашей задаче мы будем использовать датасет `movielens`, который содержит информацию о фильмах и выставленных рейтингах с сайта https://movielens.org/.

In [12]:
#Импортируем необходимые нам компоненты и считаем данные с помощью специального метода Reader:

from surprise import Dataset
from surprise import Reader
from surprise.dataset import BUILTIN_DATASETS #с помощью данного объекта мы можем использовать встроенные датасеты

data = Dataset.load_from_file(
    "data/u.data.txt",
    reader=Reader(line_format="user item rating timestamp", sep="\t"),
)

Чтобы обучать рекомендательные системы с помощью `surprise`, мы создали объект `Dataset`. Объект `surprise.dataset` — это набор данных, который содержит следующие поля в указанном порядке:

* идентификаторы пользователей,
* идентификаторы элементов,
* соответствующая оценка.
  
Преобразуем данные к формату pandas DataFrame для удобной работы с ними:

In [13]:
df = pd.DataFrame(data.raw_ratings, columns=['userId', 'movieId', 'rating', 'timestamp'])

В данных присутствуют следующие признаки:

* userId — идентификаторы пользователей сайта movielens;
* movieId — идентификаторы фильмов;
* rating — оценки фильмов, выставленные пользователями по шкале от 1 до 5;
* timestamp — время оценки фильма пользователем. Данный формат представления времени показывает, сколько секунд прошло с 1 января 1970 года.

1. Сколько уникальных фильмов в наборе данных?

In [14]:
df.movieId.value_counts().shape[0]
#1682

1682

2. Сколько уникальных пользователей в наборе данных?

In [15]:
df.userId.value_counts().shape[0]
#943

943

3. Какая оценка встречается в наборе данных чаще всего? Введите ответ в виде целого числа.

In [16]:
df.rating.mode()[0]
#4.0

4.0

Библиотека surprise очень похожа на библиотеку sklearn, и тоже позволяет разбить данные на обучающую и тестовую выборки всего одной функцией — `surprise.model_selection.train_test_split()`.

4. Разбейте данные на обучающую и тестовую выборки. Объём тестовой выборки должен составлять 25% от общего объёма данных. В качестве значения параметра `random_state` возьмите число 13.

Сколько объектов попало в тестовую выборку?

In [17]:
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(data, test_size=0.25, random_state=13)
len(testset)

25000

Импортируем функции для построения рекомендательных систем (`SVD` — для model-based-подхода и `KNNBasic` — для memory-basic-подхода) и для оценки качества результата.

In [18]:
from surprise import SVD, KNNBasic, accuracy

Теперь реализуем обычную коллаборативную фильтрацию. Выберем оценку схожести через косинусную близость и item-based-подход:

In [19]:
#Теперь реализуем обычную коллаборативную фильтрацию. Выберем оценку схожести через косинусную близость и item-based-подход:
sim_options = {
    'name': 'cosine',
    'user_based': False
}
 
knn = KNNBasic(sim_options=sim_options)

In [20]:
#Обучим алгоритм:
knn.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [21]:
#Теперь давайте посмотрим, какие рекомендации мы получили, с помощью следующей программы:
predictions = knn.test(testset)
predictions

[Prediction(uid='7', iid='633', r_ui=5.0, est=4.199452349030111, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='422', iid='287', r_ui=3.0, est=3.4703437660463736, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='804', iid='163', r_ui=3.0, est=3.5716736533692854, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='189', iid='480', r_ui=5.0, est=4.222825780855538, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='238', iid='546', r_ui=3.0, est=3.473417286928204, details={'actual_k': 17, 'was_impossible': False}),
 Prediction(uid='804', iid='216', r_ui=4.0, est=3.922551907749182, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='350', iid='204', r_ui=4.0, est=4.345238219480267, details={'actual_k': 38, 'was_impossible': False}),
 Prediction(uid='708', iid='993', r_ui=4.0, est=3.4458505791534115, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='193', iid='1078', r_ui=4.0, es

После этого можно вывести результат, записанный в переменную predictions.

Информация о каждой паре будет содержать следующие характеристики:

* uid — id пользователя;
* iid — id элемента;
* r_ui (float) — реальный рейтинг, который этот пользователь поставил этому элементу;
* est (float) — предсказанный рейтинг.

5.1 Каков реальный рейтинг, выставленный пользователем с ID 500 для фильма с ID 699?

In [22]:
import re

for i in list(range(len(list(predictions)))):
    if int(re.search(r'\d{1,5}',str(list(predictions)[i]).split('item')[1])[0])==699 and int(re.search(r'\d{1,5}',str(list(predictions)[i]).split('user')[1])[0])==500:
        print(int(re.search(r'\d{1,5}',str(list(predictions)[1]).split('r_ui')[1])[0]))
#3

3


5.2 Каков прогнозируемый рейтинг для пользователя с ID 500 и фильма с ID 699? Ответ округлите до двух знаков после точки-разделителя.

In [23]:
for i in list(range(len(list(predictions)))):
    if int(re.search(r'\d{1,5}',str(list(predictions)[i]).split('item')[1])[0])==699 and int(re.search(r'\d{1,5}',str(list(predictions)[i]).split('user')[1])[0])==500:
        print(float(re.search(r'\d.\d\d',str(list(predictions)[i]).split('est')[1])[0]))
#3.47

3.47


In [24]:
#Теперь необходимо вычислить RMSE для получившихся предсказаний:
accuracy.rmse(predictions)

RMSE: 1.0272


1.0271678039029761

Если округлить результат до сотых, получаем **1.03**.

**Итак, мы построили систему рекомендаций и даже оценили её качество. Но как же вывести рекомендации для конкретного пользователя?**

Для начала давайте оформим наши предсказания в таблицу и отсортируем их по прогнозируемому рейтингу:

In [25]:
#Для начала давайте оформим наши предсказания в таблицу и отсортируем их по прогнозируемому рейтингу:

pred = pd.DataFrame(predictions)
pred.sort_values(by=['est'],inplace=True,ascending = False)

In [26]:
#Теперь мы можем вывести рекомендуемые для конкретного пользователя фильмы, начиная от наиболее релевантного (с точки зрения рекомендаций) и заканчивая наименее релевантным.
recom = pred[pred.uid =='849']['iid'].to_list()
recom

['234', '427', '568', '174']

6. Реализуйте `user-based`-алгоритм. Какое значение RMSE получилось для коллаборативной фильтрации типа `user-based`? Ответ округлите до двух знаков после точки-разделителя.

In [27]:
#мера близости
sim_options_user_based = {
    'name': 'cosine',
    'user_based': True
}

#задаем модель 
knn_user_based = KNNBasic(sim_options=sim_options_user_based)

#обучаем
knn_user_based.fit(trainset)

#предсказываем
predictions_user_based = knn_user_based.test(testset)

#считаем метрику
accuracy.rmse(predictions_user_based).round(2)
#1.02

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.0175


1.02

7. Теперь давайте сравним полученные результаты с результатами SVD-алгоритма. Реализуйте SVD с параметрами по умолчанию.

Какое значение RMSE получилось для SVD? Ответ округлите до двух знаков после точки-разделителя.

In [28]:
#задаем модель
svd = SVD()

#обучаем
svd.fit(trainset)

#предсказываем
predictions_svd = svd.test(testset)

#считаем метрику
accuracy.rmse(predictions_svd).round(2)
#0.94

RMSE: 0.9401


0.94

8. Какой алгоритм показал наилучший результат?

*Ответ:* SVD

# Гибридные модели

Давайте на практике рассмотрим, как создать рекомендательную систему с помощью гибридного подхода.

Разумеется, можно комбинировать различные подходы самостоятельно, однако для удобства уже реализован модуль `LightFM` — установим библиотеку через следующую команду:

In [29]:
#Импортируем нужные нам функции из этой библиотеки. На этом этапе сразу же загрузим инструменты оценки модели:
from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k, recall_at_k 

c:\Users\пользователь\AppData\Local\Programs\Python\Python39\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


Работать мы будем с датасетом `goodreads_book`.

`Goodreads` — это сайт, на котором люди могут добавлять книги в каталоги, искать их, изучать аннотации и отзывы. Пользователи также могут создавать сообщества, в которых они рекомендуют друг другу различную литературу, ведут блоги и устраивают обсуждения.

In [30]:
#Подгрузим все файлы, относящиеся к этому набору данных:
ratings = pd.read_csv('data/Gooddreadbooks/ratings.zip') # Поставленные оценки
books = pd.read_csv('data/Gooddreadbooks/books.zip') # Информация о книгах
tags = pd.read_csv('data/Gooddreadbooks/tags.zip') # Информация о тегах
book_tags = pd.read_csv('data/Gooddreadbooks/book_tags.zip') # Книги с тегами 

Сначала посмотрим на набор данных books: в этих данных есть обычный id книги, а есть id книги в системе Goodreads — этот id отображён в признаке `goodreads_book_id`. В других данных `(book_tags)` указан только id книги в системе Goodreads, поэтому нам необходимо добавить туда обычный id.

1. Добавьте в набор данных `book_tags` признак с обычным id книги, используя соответствие обычного id и id в системе Goodreads.

Какой обычный id у книги, которая имеет id 5 в системе Goodreads?

In [31]:
part_books = books[['book_id', 'goodreads_book_id']]
book_tags_merged = pd.merge(left=book_tags, right=part_books, how='left', on='goodreads_book_id')
book_tags_merged[book_tags_merged['goodreads_book_id']==5]['book_id'].mean()
#18.0

18.0

2. Далее нам необходимо оставить в наборе данных `book_tags` только те записи, теги для которых есть в данных `tags`

Отфильтруйте данные таким образом, чтобы в наборе данных book_tags остались только те строки, в которых находятся теги, информация о которых есть в наборе данных tags.

Сколько объектов осталось?

In [32]:
import numpy as np

In [33]:
book_tags_merged2 = pd.merge(left=book_tags_merged, right=tags, how='left', on='tag_id')
book_tags_merged2['Nan'] = book_tags_merged2['tag_name'].isna()
book_tags_merged2 = book_tags_merged2[book_tags_merged2['Nan']==False]
book_tags_merged2 = book_tags_merged2.drop(['tag_name','Nan'], axis=1)
book_tags_merged2.shape[0]
#300738

300738

Отлично, мы подготовили информацию о тегах книг — это будет метаинформацией для построения рекомендательной системы. Теперь нам необходимо подготовить данные о взаимодействии пользователей и книг. Для этого нам понадобится файл `ratings`.

In [34]:
#Оба набора данных (и про взаимодействия, и про метаинформацию) необходимо преобразовать в разрежённые матрицы. Это можно сделать с помощью специальной функции из модуля scipy:

from scipy.sparse import csr_matrix

Нам важно преобразовать данные в специальный формат, в котором хранятся разрежённые матрицы — будем использовать формат `Compressed Sparse Row (CSR)`, подразумевающий подсчёт кумулятивной суммы количества элементов в строке вместо индексов строк.

<center> <img src=data/DST_MATH_ML_15_4_2.png alt="drawing" style="width:400px;"> </center>

Здесь хранится информация о том, сколько суммарно ненулевых элементов в данной строке и выше, индексы столбцов с ненулевыми значениями, сами значения и размерность матрицы.

В первой строке обозначено, сколько накоплено ненулевых значений (в первой строке — суммарно 1, после второй строки — суммарно 4, после третьей строки — суммарно 4, после четвёртой строки — суммарно 6).
Во второй строке показано, в каком столбце находится ненулевое значение.
В третьей строке указаны сами значения.
Осуществляем преобразование следующим образом:

In [35]:
#Осуществляем преобразование следующим образом:

ratings_matrix = csr_matrix((ratings.rating,(ratings.user_id,ratings.book_id))) # Передаём в качестве аргументов в функцию выставленный рейтинг (это будут значения матрицы), а также id пользователя и id книги (это будут индексы для строк и столбцов матрицы)

#Теперь нам необходимо составить матрицу с метаданными. В качестве индексов будут выступать id книги и id тега, и если у этой книги есть рассматриваемый тег, то на пересечении соответствующих строки и столбца будет выставлена единица.

meta_matrix  = csr_matrix(([1]*len(book_tags_merged2),(book_tags_merged2.book_id,book_tags_merged2.tag_id))) 

3. Давайте проверим, что всё получилось правильно.

Каково среднее арифметическое значений разрежённой матрицы с рейтингами? Ответ округлите до трёх знаков после точки-разделителя.

In [36]:
ratings_matrix.mean().round(3)
#0.007

0.007

In [37]:
#Отлично, данные подготовлены — теперь настало время определить модель, которую мы будем использовать. Сделаем это следующим образом:
model = LightFM(
    loss='warp-kos', # Определяем функцию потерь
    random_state=42, # Фиксируем случайное разбиение
    learning_rate=0.05, # Темп обучения
    no_components=100 # Размерность вектора для представления данных в модели
)

В качестве функции потерь мы выбрали значение 'warp', хотя, разумеется, это не единственный вариант. В модуле LightFM представлены следующие функции потерь:

* 'logistic' — логистическая функция. Полезна в случаях, когда есть как положительные, так и отрицательные взаимодействия, например 1 и -1.
* 'bpr' — байесовский персонализированный рейтинг. Можно применять, когда присутствуют только положительные взаимодействия.
* 'warp' — парный взвешенный приблизительный ранг. Используется, если необходимо повысить качество именно в верхней части списка рекомендаций.
* 'warp-kos' — модификация warp.

In [38]:
#Разобьём данные на обучающую и тестовую выборки:

train, test = random_train_test_split(
    ratings_matrix, # Общая выборка
    test_percentage=0.2, # Размер тестовой выборки
    random_state=42 # Генератор случайных чисел
)

Теперь обучим модель на наших данных о взаимодействии, также используя метаданные о книгах. Для этого воспользуемся методом `fit()`. В этот метод передадим обучающую выборку, признаки товаров — `item_features`, количество эпох обучения (сколько раз мы будем показывать модели исходный датасет, чтобы она лучше выучила данные) — `epochs`, а также параметр `verbose` для отслеживания процесса обучения:

**Обратите внимание:** из-за трудоёмкости вычислений обучение модели и оценка качества могут занимать вплоть до 15-20 минут (зависит от мощности компьютера). Не волнуйтесь, это нормальная ситуация.

**Примечание.** Если вы работаете через Google Collab, то для того, чтобы ускорить процесс обучения, вы можете в метод `fit()` передать параметр `num_threads`, в котором необходимо указать количество используемых потоков процессора. Задайте его значение, например, равным 6.

In [39]:
#model = model.fit(
    #train, # Обучающая выборка
    #item_features=meta_matrix, # Признаки товаров
    #epochs=10, # Количество эпох
    #verbose=True # Отображение обучения
#)

#код закоментирован, т.к. присутствует ошибка в библиотеке

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

: 

4. Оцените качество полученной модели с помощью функции `precision_at_k`, передав в неё три аргумента: модель, тестовые данные и обозначение метаданных `(item_features = meta_matrix)`.

*Примечание.* Процесс расчёта метрик рекомендательной системы также является довольно затратным по времени. Для ускорения этого процесса вы можете передать параметр `num_threads`, чтобы указать количество потоков процессора, используемых для вычислений.

Выведите среднее арифметическое и округлите его до двух знаков после точки-разделителя.

In [40]:
#precision_at_k(model=model, test_interactions=test, item_features = meta_matrix)

#код закоментирован, т.к. присутствует ошибка в библиотеке

#0.02

В рекомендательных системах метрики интерпретируются иначе, чем в задачах классификации. Показатели точности РС считаются хорошими, если они находятся в районе 0.1-0.3.

У нас получился не слишком высокий, но довольно неплохой результат. Чтобы его улучшить, можно попробовать следующее:

* Поработать над предобработкой данных, добавив в них дополнительную информацию о товарах. Также можно попробовать воспользоваться иным способом создания разреженной матрицы, например, форматом `coo_matrix()` или `csc_matrix()`, которые также входят в библиотеку scipy. 
* Поиграться с параметрами модели LightFM — поуправлять темпом обучения (learning_rate), размерностью вектора для представления (no_components), количеством эпох обучения (epochs) и функцией потерь (loss).

**Примечание.** Для предсказания рейтинга нового пользователя можно воспользоваться методом `predict()`:
```
scores = model.predict(<индекс интересующего пользователя>, np.arange(n_items), user_features=new_user_feature)
```

# Современные методы: глубокое обучение

Давайте разберём несложную задачу, при решении которой мы обучим настоящую нейронную сеть и используем её для создания рекомендаций.

Мы будем использовать модуль `tensorflow`, в котором реализовано много полезных методов для имплементации (внедрения) нейронных сетей. Установим его:

In [41]:
#pip install tensorflow

In [1]:
#Для начала импортируем из него функции, которые понадобятся нам для решения задачи:

from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from tensorflow.keras.models import Model


In [4]:
#Мы будем использовать данные из предыдущего юнита, но лишь те, которые содержат информацию об оценках, выставленных книгам пользователями. Загрузим данные:

df = pd.read_csv('data/Gooddreadbooks/ratings.zip')

1. Разбейте данные на обучающую и тестовую выборки в отношении 4:1. В качестве значения параметра `random_state` возьмите число 42.

Сколько объектов теперь находится в обучающей выборке?

In [5]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df,test_size=0.2, random_state=42)
train.shape[0]
#785404

785404

2. Запишите количество уникальных книг в переменную `n_books`.

Сколько в наборе данных уникальных книг?

In [6]:
n_books = df["book_id"].nunique()
print(n_books)
#10000

10000


3. Запишите количество уникальных пользователей в переменную `n_users`.

Сколько в наборе данных уникальных пользователей?

In [7]:
n_users = df["user_id"].nunique()
print(n_users)
#53424

53424


In [8]:
#В первую очередь нам необходимо создать эмбеддинги для книг и пользователей. Создаём эмбеддинги для книг:

book_input = Input(shape=[1], name="Book-Input")
book_embedding = Embedding(n_books+1, 5, name="Book-Embedding")(book_input)
book_vec = Flatten(name="Flatten-Books")(book_embedding)

In [9]:
#Сначала мы задаём размерность входного слоя (в этом параметре максимальное значение всегда равно длине вектора +1). 
# После этого определяем размер эмбеддинга — в данном случае снижаем размерность до 5. 
# Далее мы разворачиваем результат в массив с одним измерением с помощью слоя Flatten().

#Делаем то же самое для пользователей:

user_input = Input(shape=[1], name="User-Input")
user_embedding = Embedding(n_users+1, 5, name="User-Embedding")(user_input)
user_vec = Flatten(name="Flatten-Users")(user_embedding)

In [10]:
#Теперь, когда мы создали представления как для книг, так и для пользователей, нам необходимо соединить их:

conc = Concatenate()([book_vec, user_vec])

In [11]:
#Далее начинаем «собирать» нашу нейронную сеть из слоёв. 
# Dense обозначает полносвязный слой. 
# Также мы обозначаем для него количество нейронов (на первом слое будет 128 нейронов, на втором 32, 
# на последнем (выходном) -  1
# и данные (на первом слое принимаются данные от соединённых эмбеддингов, на втором – данные от 1 слоя, а в последнем  - данные от второго полносвязного слоя), которые идут на вход.

fc1 = Dense(128, activation='relu')(conc)
fc2 = Dense(32, activation='relu')(fc1)
out = Dense(1)(fc2)

In [12]:
#Собираем модель — передаём входные данные для книг и пользователей, а также архитектуру нейронной сети:

model2 = Model([user_input, book_input], out)

In [13]:
#Также нам необходимо задать алгоритм оптимизации и метрику, которую мы будем оптимизировать. В данном случае будем использовать метод adam (одна из вариаций градиентного спуска из модуля по матану) 
# и хорошо известную вам среднеквадратичную ошибку:

model2.compile(optimizer = 'adam', loss =  'mean_squared_error')

In [14]:
#Теперь будем обучать нашу модель:
#В параметр эпох передаём значение 5: у нас будет реализовано пять эпох — пять обучений нейронной сети. На каждой из эпох обновляются веса для минимизации ошибки.

history = model2.fit([train.user_id, train.book_id], train.rating, epochs=5, verbose=1)

Epoch 1/5
24544/24544 [==============================] - 533s 22ms/step - loss: 0.7938
Epoch 2/5
24544/24544 [==============================] - 502s 20ms/step - loss: 0.6874
Epoch 3/5
24544/24544 [==============================] - 487s 20ms/step - loss: 0.6575
Epoch 4/5
24544/24544 [==============================] - 531s 22ms/step - loss: 0.6341
Epoch 5/5
24544/24544 [==============================] - 506s 21ms/step - loss: 0.6135


In [15]:
#Теперь можно оценить качество:

model2.evaluate([test.user_id, test.book_id], test.rating)

6136/6136 [==============================] - 29s 5ms/step - loss: 0.7093


0.7093052268028259

**Примечание.** К сожалению, результаты этого алгоритма нельзя зафиксировать стандартным ramdom_state, к которому мы привыкли: применяемые методы не используют такой параметр. Поэтому мы опустим здесь сравнение результатов, однако посмотрим, как можно настроить нейронную сеть.

In [16]:
# Обычно для улучшения качества модели каким-то образом модифицируют нейронную сеть: дополняют её, увеличивают время обучения. 
# Добавим ещё один полносвязный слой с восемью нейронами после полносвязного слоя с 32 нейронами. Обучим нейронную сеть, реализовав десять эпох:
fc1 = Dense(128, activation='relu')(conc)
fc2 = Dense(32, activation='relu')(fc1)
fc3 = Dense(8, activation='relu')(fc2)
out = Dense(1)(fc3)

model2 = Model([user_input, book_input], out)
model2.compile('adam', 'mean_squared_error')
result = model2.fit([train.user_id, train.book_id], train.rating, epochs=10, verbose=1)
model2.evaluate([test.user_id, test.book_id], test.rating)

Epoch 1/10
24544/24544 [==============================] - 513s 21ms/step - loss: 0.6355
Epoch 2/10
24544/24544 [==============================] - 648s 26ms/step - loss: 0.5869
Epoch 3/10
24544/24544 [==============================] - 522s 21ms/step - loss: 0.5691
Epoch 4/10
24544/24544 [==============================] - 506s 21ms/step - loss: 0.5534
Epoch 5/10
24544/24544 [==============================] - 508s 21ms/step - loss: 0.5386
Epoch 6/10
24544/24544 [==============================] - 514s 21ms/step - loss: 0.5261
Epoch 7/10
24544/24544 [==============================] - 489s 20ms/step - loss: 0.5151
Epoch 8/10
24544/24544 [==============================] - 1966s 80ms/step - loss: 0.5054
Epoch 9/10
24544/24544 [==============================] - 508s 21ms/step - loss: 0.4971
Epoch 10/10
6136/6136 [==============================] - 28s 5ms/step - loss: 0.7735


0.7734776139259338

Качество получившейся модели не будет выше качества предыдущей, так как усложнение сети или увеличение количества эпох не всегда даёт высокое качество. Здесь главное, что вы научились корректировать архитектуру нейронной сети.